In [1]:
%matplotlib
#%matplotlib inline
import os
import csv
import fnmatch
import numpy as np
import datetime
import re 
import pandas as pd
import matplotlib.pyplot as plt
import math
import pywt
import itertools
from scipy import stats

pd.options.mode.use_inf_as_na = True

Using matplotlib backend: TkAgg


In [2]:
TimeDwellOrig = 800
TimeFixation = 300

In [3]:
pupilTotal_Difficult = list()
pupilTotal_Medium = list()
pupilTotal_Easy = list()

In [4]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [5]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [6]:
def OptiKeyTypingTime(UserKeys):
    
    TimeTyping = dict()
    
    time1, t1, t2 = UserKeys[0][0].partition('+')
    startTime = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    time2, t1, t2 = UserKeys[-1][0].partition('+')
    endTime = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
    TimeTyping['startTime'] = startTime
    TimeTyping['endTime'] = endTime
    
    return TimeTyping

In [7]:
def FindTrialEndTimes(KeysSelected, timeTyping):
    # function to find start and end of tasks in experiments
    timeStartEnd = list() # format of this list will be: [startTime1, endTime1/startTime2, endTime2/startTime3, ..., endTimeN]
    
    timeStartEnd.append(timeTyping['startTime'])
    
    nTrial = 1
    
    for keys in KeysSelected:
        
        if keys[1] == 'NextPhrase':
            time1, t1, t2 = keys[0].partition('+')
            endTimeTrial = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            timeStartEnd.append(endTimeTrial)
    
    
    timeStartEnd.append(timeTyping['endTime'])
    
    
    return timeStartEnd

In [8]:
# function to convert list of date and time into datetime format list
def timeConversion(timeStrList):
    timeList = list()
    for time in timeStrList:
        time1, t1, t2 = time.partition('+')
        timeList.append(datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f"))
    return timeList

In [9]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [10]:
def CreateTimeEpochsOfTrials(TimeStartEndMixed, UserKeys):
    # function to use list of mixed start and end times of trials and keys looked at by user to create trial epochs
    
    TimeEpochTrial = dict()
    TimeEpochTrial['Start'] = list()
    TimeEpochTrial['End'] = list()
    
    # Create list of times in userKeys to be able to use function 'nearestTimePoint'
    UserKeysStrTimes = [item3[0] for item3 in UserKeys]
    UserKeysTimes = timeConversion(UserKeysStrTimes)
    
    Flag_FoundSleepKey = 0 # Flag to indicate finding sleep key
    
    n = -1
    for time in TimeStartEndMixed:
        n = n + 1
        Flag_FoundSleepKey = 0
        
        if n == 0: # first time is only start time for the first trial
            TimeEpochTrial['Start'].append(time)
            continue
        elif n == len(TimeStartEndMixed)-1: # last time is only the end time for last trial
            
            TimeEpochTrial['End'].append(time)
            
        else: # the middle elements need to be divided into start and end
            TimeEpochTrial['End'].append(time)
            
            timeCheck = time
            
            # find the time in userkeys. Keep going to the previous element till you reach start of selection of
            # nextPhrase key
            while Flag_FoundSleepKey < 1:
                
                nearestToTrialStartTime, nearestToTrialStartInd = nearestTimePoint(UserKeysTimes, timeCheck)
                indCheck = nearestToTrialStartInd
                
                if 'NextPhrase' not in UserKeys[indCheck][1]:
                    TimeEpochTrial['Start'].append(nearestToTrialStartTime)
                    Flag_FoundSleepKey = 1
                    break
                else:
                    indCheck = indCheck - 2 # 2 added instead of 1, to allow nearestTimePoint to find the one before this
                    timeCheck = UserKeysTimes[indCheck]
                    
                
    return TimeEpochTrial      
            

In [11]:
def DwellTimeForBaseline(UserKeys_wDwellTime):
    
    DwellTime = list()
    
    for key in UserKeys_wDwellTime:
        if key[1] == 'NextPhrase':
            #print('NextPhrase found at ', key[2])
            if key[2] == 1:
                DwellTime.append(key[3])
                
    return DwellTime

In [12]:
def Combine2ColumnsTo1GazeLog(GazeLog, Column_1, Column_2):
    
    JoinedList = list()
    
    Column_beforeDecimal = [item4[Column_1] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    Column_afterDecimal = [item4[Column_2] if 'Invalid' not in item4 else 'nan' for item4 in GazeLog]
    
    
    for i in range(0, len(Column_beforeDecimal)):
        if 'Valid' not in Column_beforeDecimal[i] and 'Valid' not in Column_afterDecimal[i]:
            if 'nan' not in Column_beforeDecimal[i] and 'nan' not in Column_afterDecimal[i]:
                JoinedList.append(float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]))
            else:
                JoinedList.append(np.nan)
        else:
            # Rarely, the pupil size is a whole number
            JoinedList.append(np.nan) # we will ignore the row, since there is no way of automatically knowing which - 
            # right or left eye has whole number pupil size
            
    return JoinedList

In [13]:
def Convert2ColumnsToFormPupilSizes(GazeLog):
    # function to convert pupilsizes from 2 columns for every pupil due to comma use instead of decimal, 
    # to proper pupil sizes
    
    PupilLogL = Combine2ColumnsTo1GazeLog(GazeLog, -5, -4)
    PupilLogR = Combine2ColumnsTo1GazeLog(GazeLog, -2, -1)
            
    # if one of the pupils are nan, the other one is converted too
    nPupil = -1
    for pupilL in PupilLogL:
        nPupil = nPupil + 1
        if np.isnan(pupilL):
            if nPupil < len(PupilLogR):
                if not np.isnan(PupilLogR[nPupil]):
                    PupilLogR[nPupil] = np.nan
            else:
                del PupilLogL[len(PupilLogR):]
                
    nPupil = -1
    for pupilR in PupilLogR:
        nPupil = nPupil + 1
        if np.isnan(pupilR):
            if nPupil < len(PupilLogL):
                if not np.isnan(PupilLogL[nPupil]):
                    PupilLogL[nPupil] = np.nan
            else:
                del PupilLogR[len(PupilLogL):]
                
    #print(len(PupilLogL), len(PupilLogR))
    
    return PupilLogL, PupilLogR

In [14]:
def PupilSizeFromTrialTimes(TimeTrial, TimeGazeLog, TimeInternalGazeLog, PupilSizeLogL, PupilSizeLogR):
    # find pupil sizes from the start and end time given
    
    # find start and end time in gazeLog
    timeStart, timeStartInd = nearestTimePoint(TimeGazeLog, TimeTrial[0])
    timeEnd, timeEndInd = nearestTimePoint(TimeGazeLog, TimeTrial[1])
    
    pupilSize_TrialL = PupilSizeLogL[timeStartInd: timeEndInd]
    pupilSize_TrialR = PupilSizeLogR[timeStartInd: timeEndInd]
    
    TimeInternal_Trial = TimeInternalGazeLog[timeStartInd: timeEndInd]
    
    TimeGaze_Trial = TimeGazeLog[timeStartInd: timeEndInd]
    
    return pupilSize_TrialL, pupilSize_TrialR, TimeGaze_Trial, TimeInternal_Trial
    

In [15]:
def filterBlinks(pupilData, timeListComplete):
    # filter any blinks and nan values lasting around 250ms (on average)
    # http://faculty.washington.edu/chudler/facts.html
   
    # blink is every nan value in the range of 100-400ms 
    # 250 ms (23 samples) before and after the blink will also be removed
    extraBlinkSamples = 23    
    
    # remove single missing data, that are due to hardware error
    missingVal_Single = np.argwhere(np.isnan(pupilData))
    missingVal_Single = list(itertools.chain.from_iterable(missingVal_Single)) # flatten the list
    missingVal_SingleDifference = [t - s for s, t in zip(missingVal_Single, missingVal_Single[1:])] # find difference 
    # between consecutive elements
    missingVal_SingleDifference.insert(0, missingVal_Single[0]) # insert the first blink index in the beginning of list
    
    # the list missingVal_SingleDifference contains the index of the first blink, followed by the difference in the index to 
    # the next nan value
    
    # first remove the single nan values, which are missing data
    eyeTracker_missingData = list() # list with index of single missing data  
    valInd = -1

    for val in missingVal_SingleDifference:
        valInd = valInd + 1
        if valInd == 0:
            continue
        if val != 1:
            if missingVal_SingleDifference[valInd-1] !=1: # if there are 2 consecutive missing values (denoted by 2 consecutive
                # non 1 numbers, they are added to the list of eyeTracker_missingData)
                eyeTracker_missingData.append(sum(missingVal_SingleDifference[:valInd]))
                
    # remove single missing values from pupil data
    pupilData_woSingleMissingData0 = [pupilData[ind] if ind not in eyeTracker_missingData else [] 
                                     for ind in range(0, len(pupilData))]
    pupilData_woSingleMissingData = [x for x in pupilData_woSingleMissingData0 if x]
    
    # remove the times for single missing values in pupil data
    timeList_woSingleMissingData0 = [timeListComplete[ind] if ind not in eyeTracker_missingData else [] 
                                     for ind in range(0, len(timeListComplete))]
    timeList_woSingleMissingData = [x for x in timeList_woSingleMissingData0 if x]
    
#     print(len(timeList_woSingleMissingData))
    
    
    
    
    # find the nan values again from pupilData_woSingleMissingData
    missingVal_Rest = np.argwhere(np.isnan(pupilData_woSingleMissingData))
    missingVal_Rest = list(itertools.chain.from_iterable(missingVal_Rest))
    missingVal_RestDifference = [t - s for s, t in zip(missingVal_Rest, missingVal_Rest[1:])]
    missingVal_RestDifference.insert(0, missingVal_Rest[0])
    
    
    # compile and create list of start and end of blinks
    blink_missingData = dict()
    blink_missingData['Start'] = list()
    blink_missingData['End'] = list()
    
    valInd = -1
    for val in missingVal_RestDifference:
        valInd = valInd + 1
        if val > 1:
            #print('value', val)
            # instead of appending the actual index of blink start, since 250ms before and after the blink need to be
            # removed, it is also appended here.
            
            # just make sure that the additional samples do not make the index of blink go in negative
            if sum(missingVal_RestDifference[:valInd+1])-extraBlinkSamples > 0:
                
                blink_missingData['Start'].append(sum(missingVal_RestDifference[:valInd+1])-extraBlinkSamples)
            else:
                blink_missingData['Start'].append(0)
            
            if valInd == 0:
                continue
                
            # make sure that the additional samples do not increase the index to more than the length of the pupilData
            if sum(missingVal_RestDifference[:valInd])+extraBlinkSamples < len(pupilData_woSingleMissingData):
                blink_missingData['End'].append(sum(missingVal_RestDifference[:valInd])+extraBlinkSamples)
            else:
                blink_missingData['End'].append(len(pupilData_woSingleMissingData)-1)
            #print('end', sum(missingVal_RestDifference[:valInd]))
      
    # add the last blink index
    # make sure that the additional samples do not increase the index to more than the length of the pupilData
    if sum(missingVal_RestDifference)+extraBlinkSamples < len(pupilData_woSingleMissingData):
        blink_missingData['End'].append(sum(missingVal_RestDifference)+extraBlinkSamples)
    else:
        blink_missingData['End'].append(len(pupilData_woSingleMissingData)-1)
      
    
    # print start and end values
    #for ind in range(0,len(blink_missingData['Start'])):
    #    print(blink_missingData['Start'][ind]+23, blink_missingData['End'][ind]-23)
        
        
        
    # need to create a list containing indexes that are to be removed
    blinkIndexList = list()
    
    #print(len(blink_missingData['Start']), len(blink_missingData['End']))
    
    
    # remove blinks and additional data from pupil data to get filtered data
    for indInd in range(0, len(blink_missingData['Start'])):
        blinkIndexList.append(range(blink_missingData['Start'][indInd], blink_missingData['End'][indInd]+1))
    # flatten the list
    blinkIndexList = list(itertools.chain.from_iterable(blinkIndexList))
    
    
    ##print(len(pupilData_woSingleMissingData))
    
    pupilData_woRestMissingData0 = [pupilData_woSingleMissingData[ind] if ind not in blinkIndexList else [] 
                                     for ind in range(0, len(pupilData_woSingleMissingData))]
    #for i in enumerate(pupilData_woRestMissingData0):
    #    print(i)
    pupilData_filter = [x for x in pupilData_woRestMissingData0 if x]
    
    #for i in enumerate(pupilData_filter):
    #    print(i)
        
    # remove the times for single missing values in pupil data
    timeList_woRestMissingData0 = [timeList_woSingleMissingData[ind] if ind not in blinkIndexList else [] 
                                     for ind in range(0, len(timeList_woSingleMissingData))]
    time_filter = [x for x in timeList_woRestMissingData0 if x]
    
    #print(len(pupilData_filter))
        
    if np.nan in pupilData_filter:
        print('nan values still present in pupil data')
        #for i in enumerate(pupilData_woSingleMissingData):
        #    print(i)
        
    
    return pupilData_filter, time_filter, blink_missingData

In [16]:
def hampel(vals_orig, k, sd):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''
    # Obtained from: https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-
    # hampel-function-faster
    
    #plt.plot(vals_orig)
    
    #Make copy so original not edited
    vals = pd.DataFrame(vals_orig)      
    #print(vals.isnull().any())
    vals0 = vals.replace([np.inf, -np.inf], np.nan)
    #vals = vals0.astype(float).fillna(method = 'backfill') # linear interpolation instead 
    #print(vals)
    vals = vals0.astype(float).interpolate('linear', limit_direction = 'both') # linear interpolation instead of 
    # simply copying the previous value --\ linear interpolation than cubic to not add any patterns in the data, limit direction
    # set to both, to interpolate the nan values occuring from the start of the series
    
    L= 1.4826
    rolling_median = vals.rolling(window=k, min_periods=1, center=True).median()
    
    #print(rolling_median)
    difference = np.abs(rolling_median-vals)
    median_abs_deviation = difference.rolling(k).median()
    threshold = sd * L * median_abs_deviation
    outlier_idx = difference>threshold
    vals[outlier_idx] = rolling_median[outlier_idx]
    #print(vals)
    #print('datatype', vals.dtypes)
    #print(vals.isnull().any())
    #vals.plot()
    return(vals)

In [17]:
def plotPupilSize(pupilData, timeData, TrialNumber):
    
    dataLenEqualizer = min(min(len(pupilData['Left']), len(pupilData['Right'])), len(timeData))
    
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1)
    ax.plot(timeData[0:dataLenEqualizer], pupilData['Left'][0:dataLenEqualizer], 'b')
    ax.plot(timeData[0:dataLenEqualizer], pupilData['Right'][0:dataLenEqualizer], 'r')
    
    ax.set_ylabel('Relative pupil size [in mm]')

    ax.set_title(TrialNumber)


In [34]:
def FindAndPlotPupilSizeForEpoch(GazeLog, TimeEpochTrial, DwellTimes_ForBaseline):
    # function that uses the list of start and end trial times to find the pupil sizes for those trials and plots them
    
    # first create a list of times in gaze log
    timeStrGazeLog = [item3[0] for item3 in GazeLog]
    # convert the list of strings to datetime formats
    timeGazeLog = timeConversion(timeStrGazeLog)
    
    # internal time, to depict seconds
    timeInternalGazeLog = [float(item3[1]) for item3 in GazeLog]
    
    # extract pupil sizes in decimals from the strange 2 columns for every pupil
    pupilLogL, pupilLogR = Convert2ColumnsToFormPupilSizes(GazeLog)
    
    pupilL_avg = list()
    pupilR_avg = list()
    timeOfGaze_TrialList = list()
    pupil_AvgPartsList = list()
    
    # for every epoch, plot the pupil size
    for trialNr in range(0, len(timeEpochTrial['Start'])):
        if trialNr == 0:
            continue
        print(trialNr)
        # find pupil sizes for the trial
        pupilSizeL_Trial, pupilSizeR_Trial, timeGaze_Trial, timeInternal_Trial = PupilSizeFromTrialTimes(
            [TimeEpochTrial['Start'][trialNr], TimeEpochTrial['End'][trialNr]], timeGazeLog, 
                                timeInternalGazeLog, pupilLogL, pupilLogR)
        
        pupilSize_Trial = dict()
        pupilSize_Filter = dict()
        pupilSize_woBlink = dict()
        
        # find difference in consecutive elements of internal time
        timeInternalDifference = [t - s for s, t in zip(timeInternal_Trial, timeInternal_Trial[1:])]
        # divide by 1000 to make it s
        timeOfGaze_Trial = [sum(timeInternalDifference[:i])/1000000 for i in range(1,len(timeInternalDifference))]

        # some trials were skipped, because the sentence was written before. If the time of trial is less than
        # 10s, the trial is skipped
        if timeOfGaze_Trial[-1] < 20:
            print('trial number ', trialNr+1, 'with', timeOfGaze_Trial[-1], 's will be skipped')
            continue
        
        pupilSize_Trial['Left'] = pupilSizeL_Trial
        pupilSize_Trial['Right'] = pupilSizeR_Trial
        
        #if trialNr == 4:
        #    for i in range(0, len(pupilSizeL_Trial)):
        #        print(pupilSizeL_Trial[i], pupilSizeR_Trial[i])
            
        #print('Trial', len(pupilSizeL_Trial), len(pupilSizeR_Trial))
        
        # filter the blinks
        pupilSizeL_woBlink, time_filter, missingPupilData = filterBlinks(
            pupilSizeL_Trial, timeGaze_Trial)
        
        pupilSizeR_woBlink, time_filter, missingPupilData = filterBlinks(
            pupilSizeR_Trial, timeGaze_Trial)
        

        
        #print(index_blinkEndL)
        #print(index_blinkEndR)
        pupilSize_woBlink['Left'] = pupilSizeL_woBlink
        pupilSize_woBlink['Right'] = pupilSizeR_woBlink
        
        #print('After blink', len(pupilSizeL_woBlink), len(pupilSizeR_woBlink))
        # Hampel filter to remove the outliers
        winSize = 25
        pupilSizeL_filter = hampel(pupilSizeL_woBlink, winSize, 3)
        pupilSizeR_filter = hampel(pupilSizeR_woBlink, winSize, 3)

        pupilSize_Filter['Left'] = pupilSizeL_filter.values.tolist()
        pupilSize_Filter['Right'] = pupilSizeR_filter.values.tolist()
        
        pupilSizeL_filterList = [i[0] for i in pupilSizeL_filter.values]
        pupilSizeR_filterList = [i[0] for i in pupilSizeR_filter.values]
        
        #print('filter', len(pupilSizeL_filterList), len(pupilSizeR_filterList))
        RLCorrelation = np.corrcoef(pupilSizeL_filterList, pupilSizeR_filterList)
        
        if RLCorrelation[0][1] < 0.8:
            print(RLCorrelation[0][1])
            print('CORRELATION BETWEEN RIGHT AND LEFT IS NOT GOOD. TRIAL MUST BE REMOVED')
        
        # Relative Pupil Size Calculation 
        # First find baseline pupil size, which is the time when looking at NextPhrase key
        Samples_ForBaseline = int((int(DwellTimes_ForBaseline[trialNr-1][:-2])*90)/1000) # Number of samples of looking at key depend on
        
        #print(DwellTimes_ForBaseline[trialNr-1])
        
        # dwell time
        pupilL_baseline = np.mean(pupilSizeL_filterList[0:Samples_ForBaseline])
        pupilR_baseline = np.mean(pupilSizeR_filterList[0:Samples_ForBaseline])
        
        pupilL_Relative = [pupil - pupilL_baseline for pupil in pupilSizeL_filterList]
        pupilR_Relative = [pupil - pupilR_baseline for pupil in pupilSizeR_filterList]
        
        
        # Divide Trial into 4 parts
        samplesInOneQuarter = int(len(pupilL_Relative)/4)
        
        pupilL_AvgPart1 = np.mean(pupilL_Relative[0:samplesInOneQuarter-1])
        pupilL_AvgPart2 = np.mean(pupilL_Relative[samplesInOneQuarter:int(2*samplesInOneQuarter)-1])
        pupilL_AvgPart3 = np.mean(pupilL_Relative[int(2*samplesInOneQuarter):int(3*samplesInOneQuarter)-1])
        pupilL_AvgPart4 = np.mean(pupilL_Relative[int(3*samplesInOneQuarter):])
        
        pupilR_AvgPart1 = np.mean(pupilR_Relative[0:samplesInOneQuarter-1])
        pupilR_AvgPart2 = np.mean(pupilR_Relative[samplesInOneQuarter:int(2*samplesInOneQuarter)-1])
        pupilR_AvgPart3 = np.mean(pupilR_Relative[int(2*samplesInOneQuarter):int(3*samplesInOneQuarter)-1])
        pupilR_AvgPart4 = np.mean(pupilR_Relative[int(3*samplesInOneQuarter):])
        
        pupilL_AvgParts = [pupilL_AvgPart1, pupilL_AvgPart2, pupilL_AvgPart3, pupilL_AvgPart4]
        pupilR_AvgParts = [pupilR_AvgPart1, pupilR_AvgPart2, pupilR_AvgPart3, pupilR_AvgPart4]
        
        pupilAvg_AvgParts = [(pupilL_AvgParts[ind] + pupilR_AvgParts[ind])/2 for ind in range(0, len(pupilL_AvgParts))]
        
        pupilSize_Relative = dict()
        pupilSize_Relative['Left'] = pupilL_Relative
        pupilSize_Relative['Right'] = pupilR_Relative
        
        #plotPupilSize(pupilSize_Relative, timeOfGaze_Trial, trialNr)
        
        pupil_AvgPartsList.append(pupilAvg_AvgParts)
        
    return pupil_AvgPartsList

In [66]:
subjName = r'C:\DTU\Data\201812_ExptToCheckMovementEffect\Data'
j = 0
flagFirstSubj = 0
pupilData = dict()
pupilData['RLCorrelation'] = []


# extract self-reported scores list and LIX score of given sentence
file_name = r'C:/DTU/Data/201812_ExptToCheckMovementEffect/Data/Scores.xlsx'

testNr = 'Test_wChinRest'
for root, dirs, subfolder in os.walk(subjName):
    if not dirs and testNr in root and 'sa' in root:
        
        if 'tb' in root or 'trial' in root:
            continue
            
        userKeys = None
        gazeLog = None
        keysSelected = None
        
        for file in subfolder:
            if fnmatch.fnmatch(file, 'user_looks*'):
                try:
                    
                    fUserKey = open(root + '\\' + file, encoding='utf-8')
                    readerUserKey = csv.reader(fUserKey)
                    userKeys = list(readerUserKey)
                    
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        
                        fUserKey.close()
                    else:
                        print('error in opening the user looks at log file')
            
            elif fnmatch.fnmatch(file, 'KeySelection*'):
                try:
                    
                    fKeysSelected = open(root + '\\' + file, encoding='utf-8')
                    readerKeysSelected = csv.reader(fKeysSelected)
                    keysSelected = list(readerKeysSelected)
                    
                    keysSelected.remove(keysSelected[0])
                except:
                    if fKeysSelected is not None:
                        
                        fKeysSelected.close()
                    else:
                        print('error in opening the KeySelection log file')
            
            elif fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8')
                    readerGazeLog = csv.reader(fGazeLog)
                    gazeLog = list(readerGazeLog)
                    
                    gazeLog.remove(gazeLog[0]) # would not matter much even if the first row was not labels
                    gazeLog.remove(gazeLog[-1])

                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the gaze log file')
            else:
                continue
            
                # if all these lists exist
            if userKeys is None or keysSelected is None or gazeLog is None:
                continue
            else:
                
                a = re.compile('(?<=ExptToCheckMovementEffect\\\\Data\\\\)(.*)(?=\\\\2018-1)')
                subjName = a.findall(root)[0]
                print(subjName)
                
                if subjName == 'sa\Test_woChinRest\p2':
                    userKeys = userKeys[:-1]
                
                sheet_to_df_map = pd.read_excel(file_name, sheet_name=subjName[0:2])
                columnName = testNr + '_SumOfScores'
                
                # find total of difficulty score
                scoresDifficulty = sheet_to_df_map[columnName]
                scoresDifficulty = scoresDifficulty[1:]
                
                print(scoresDifficulty)
                
                # fix userKeys due to comma related file changes
                userKeys_new = FixUserKeys(userKeys)
                
                # find dwell time of typing
                userKeys_wDwellTime = ComputeDwellTime(userKeys_new)
                
                # find start time of typing
                timeTyping = OptiKeyTypingTime(userKeys_wDwellTime)
                
                # for some of the subjects, the data was not completely collected
                if subjName == 'sa\Test_woChinRest\p1' or subjName == 'rh\Test_wChinRest\p1':
                    del keysSelected[-1]
                
                # divide complete data into epochs of phrases
                timeStartEndMixed = FindTrialEndTimes(keysSelected, timeTyping)
                
                # create trial time epoch using the list of start/end times of trial and userKeys, to make sure that 
                # Sleep is completely there in every trial, to allow for baseline
                timeEpochTrial = CreateTimeEpochsOfTrials(timeStartEndMixed, userKeys_wDwellTime)
                #print(timeEpochTrial)
                
                dwellTimes_ForBaseline = DwellTimeForBaseline(userKeys_wDwellTime)
                
                # find and plot pupil size for every trial
                pupilAvg_AvgPartsList = FindAndPlotPupilSizeForEpoch(gazeLog, timeEpochTrial, dwellTimes_ForBaseline)
                
                
                
                
                if 'sa\Test_woChinRest' in subjName or 'rh\Test_wChinRest' in subjName:
                    if 'p1' in root:
                        pupilAvg_AvgPartsList1 = pupilAvg_AvgPartsList
                        
                    else:
                        if 'sa\Test_woChinRest' in subjName:
                            pupilAvg_AvgPartsList2 = pupilAvg_AvgPartsList[1:]
                            

                        else:
                            pupilAvg_AvgPartsList2 = pupilAvg_AvgPartsList
                            

sa\Test_wChinRest
1     7
2     5
3     9
4     7
5    12
6    12
7    10
8     8
9    11
Name: Test_wChinRest_SumOfScores, dtype: int64
1
2
3
4
5
6
7
8
9


In [67]:
if 'sa\Test_woChinRest' in subjName or 'rh\Test_wChinRest' in subjName:
    print('Addition of lists')
    pupilAvg_AvgPartsList = pupilAvg_AvgPartsList1 + pupilAvg_AvgPartsList2
    

pupil_Part1 = [item[0] for item in pupilAvg_AvgPartsList]
pupil_Part2 = [item[1] for item in pupilAvg_AvgPartsList]
pupil_Part3 = [item[2] for item in pupilAvg_AvgPartsList]
pupil_Part4 = [item[3] for item in pupilAvg_AvgPartsList]
    
    
pupil_Difficult_Part1 = list()
pupil_Medium_Part1 = list()
pupil_Easy_Part1 = list()

pupil_Difficult_Part2 = list()
pupil_Medium_Part2 = list()
pupil_Easy_Part2 = list()

pupil_Difficult_Part3 = list()
pupil_Medium_Part3 = list()
pupil_Easy_Part3 = list()

pupil_Difficult_Part4 = list()
pupil_Medium_Part4 = list()
pupil_Easy_Part4 = list()


for ind in range(0, len(scoresDifficulty)):
    #print(scoresDifficulty.values[ind])
    if scoresDifficulty.values[ind] > 12:
        pupil_Difficult_Part1.append(pupil_Part1[ind])
        pupilTotal_Difficult_Part1.append(pupil_Part1[ind])
        pupil_Difficult_Part2.append(pupil_Part2[ind])
        pupilTotal_Difficult_Part2.append(pupil_Part2[ind])
        pupil_Difficult_Part3.append(pupil_Part3[ind])
        pupilTotal_Difficult_Part3.append(pupil_Part3[ind])
        pupil_Difficult_Part4.append(pupil_Part4[ind])
        pupilTotal_Difficult_Part4.append(pupil_Part4[ind])
        
        
    elif scoresDifficulty.values[ind] > 7:
        pupil_Medium_Part1.append(pupil_Part1[ind])
        pupilTotal_Medium_Part1.append(pupil_Part1[ind])
        pupil_Medium_Part2.append(pupil_Part2[ind])
        pupilTotal_Medium_Part2.append(pupil_Part2[ind])
        pupil_Medium_Part3.append(pupil_Part3[ind])
        pupilTotal_Medium_Part3.append(pupil_Part3[ind])
        pupil_Medium_Part4.append(pupil_Part4[ind])
        pupilTotal_Medium_Part4.append(pupil_Part4[ind])
        
    else:
        pupil_Easy_Part1.append(pupil_Part1[ind])
        pupilTotal_Easy_Part1.append(pupil_Part1[ind])
        pupil_Easy_Part2.append(pupil_Part2[ind])
        pupilTotal_Easy_Part2.append(pupil_Part2[ind])
        pupil_Easy_Part3.append(pupil_Part3[ind])
        pupilTotal_Easy_Part3.append(pupil_Part3[ind])
        pupil_Easy_Part4.append(pupil_Part4[ind])
        pupilTotal_Easy_Part4.append(pupil_Part4[ind])

#fig = plt.figure()
#ax = fig.add_subplot(1,1,1)
#ax.plot(pupilAvg)
#ax.plot(range(1,len(pupil_Difficult)+1), pupil_Difficult, 'r', marker = 'o', linestyle = '-', label = 'difficult')
#ax.plot(range(1,len(pupil_Medium)+1), pupil_Medium, 'b', marker = 's', linestyle = '-', label = 'medium')
#ax.plot(range(1,len(pupil_Easy)+1), pupil_Easy, 'c', marker = 'd', linestyle = '-', label = 'easy')


#ax.plot(range(1,len(scoresDifficulty)+1), scoresDifficulty/100, 'b', label = 'difficulty')
#ax.plot(range(1, len(timeOfTrialList)+1), timeOfTrialListPlot, 'c', label = 'time' )
#ax.set_title(subjName)
#ax.legend()
             
#print(ipaList)
#print(scoresDifficulty)


In [150]:
print(stats.ttest_ind(pupilTotal_Difficult,pupilTotal_Medium))
print(stats.ttest_ind(pupilTotal_Difficult,pupilTotal_Easy))
print(stats.ttest_ind(pupilTotal_Easy,pupilTotal_Medium))

Ttest_indResult(statistic=0.5950996582177246, pvalue=0.5649994869607455)
Ttest_indResult(statistic=2.016995595396921, pvalue=0.0713371592961708)
Ttest_indResult(statistic=-0.41729265646520425, pvalue=0.6852817328789658)


In [71]:
#part1
print('1', stats.ttest_ind(pupilTotal_Difficult_Part1,pupilTotal_Medium_Part1))
print('2', stats.ttest_ind(pupilTotal_Difficult_Part1,pupilTotal_Easy_Part1))
print('3', stats.ttest_ind(pupilTotal_Easy_Part1,pupilTotal_Medium_Part1))

#part2
print('4', stats.ttest_ind(pupilTotal_Difficult_Part2,pupilTotal_Medium_Part2))
print('5', stats.ttest_ind(pupilTotal_Difficult_Part2,pupilTotal_Easy_Part2))
print('6', stats.ttest_ind(pupilTotal_Easy_Part2,pupilTotal_Medium_Part2))

#part3
print('7', stats.ttest_ind(pupilTotal_Difficult_Part3,pupilTotal_Medium_Part3))
print('8', stats.ttest_ind(pupilTotal_Difficult_Part3,pupilTotal_Easy_Part3))
print('9', stats.ttest_ind(pupilTotal_Easy_Part3,pupilTotal_Medium_Part3))


#part4
print('10', stats.ttest_ind(pupilTotal_Difficult_Part4,pupilTotal_Medium_Part4))
print('11', stats.ttest_ind(pupilTotal_Difficult_Part4,pupilTotal_Easy_Part4))
print('12', stats.ttest_ind(pupilTotal_Easy_Part4,pupilTotal_Medium_Part4))


# easy
print('13', stats.ttest_ind(pupilTotal_Easy_Part1,pupilTotal_Easy_Part2))
print('14', stats.ttest_ind(pupilTotal_Easy_Part1,pupilTotal_Easy_Part3))
print('15', stats.ttest_ind(pupilTotal_Easy_Part1,pupilTotal_Easy_Part4))
print('16', stats.ttest_ind(pupilTotal_Easy_Part2,pupilTotal_Easy_Part3))
print('17', stats.ttest_ind(pupilTotal_Easy_Part2,pupilTotal_Easy_Part4))
print('18', stats.ttest_ind(pupilTotal_Easy_Part3,pupilTotal_Easy_Part4))

# medium
print('19', stats.ttest_ind(pupilTotal_Medium_Part1,pupilTotal_Medium_Part2))
print('20', stats.ttest_ind(pupilTotal_Medium_Part1,pupilTotal_Medium_Part3))
print('21', stats.ttest_ind(pupilTotal_Medium_Part1,pupilTotal_Medium_Part4))
print('22', stats.ttest_ind(pupilTotal_Medium_Part2,pupilTotal_Medium_Part3))
print('23', stats.ttest_ind(pupilTotal_Medium_Part2,pupilTotal_Medium_Part4))
print('24', stats.ttest_ind(pupilTotal_Medium_Part3,pupilTotal_Medium_Part4))


# difficult
print('25', stats.ttest_ind(pupilTotal_Difficult_Part1,pupilTotal_Difficult_Part2))
print('26', stats.ttest_ind(pupilTotal_Difficult_Part1,pupilTotal_Difficult_Part3))
print('27', stats.ttest_ind(pupilTotal_Difficult_Part1,pupilTotal_Difficult_Part4))
print('28', stats.ttest_ind(pupilTotal_Difficult_Part2,pupilTotal_Difficult_Part3))
print('29', stats.ttest_ind(pupilTotal_Difficult_Part2,pupilTotal_Difficult_Part4))
print('30', stats.ttest_ind(pupilTotal_Difficult_Part3,pupilTotal_Difficult_Part4))




1 Ttest_indResult(statistic=-1.0696403057017287, pvalue=0.28921262865813135)
2 Ttest_indResult(statistic=0.044449822205945996, pvalue=0.9647302539565396)
3 Ttest_indResult(statistic=-1.231538985966445, pvalue=0.2223634647575932)
4 Ttest_indResult(statistic=-0.8344306596878612, pvalue=0.4074620949781256)
5 Ttest_indResult(statistic=-0.7562252867819801, pvalue=0.45320975139779984)
6 Ttest_indResult(statistic=-0.09953408804485739, pvalue=0.921007079909333)
7 Ttest_indResult(statistic=-0.28602804755113254, pvalue=0.7758757731750251)
8 Ttest_indResult(statistic=-0.0825502957293609, pvalue=0.9345524139005935)
9 Ttest_indResult(statistic=-0.2202583053584867, pvalue=0.826329925051366)
10 Ttest_indResult(statistic=-1.2867630161383408, pvalue=0.20328908531097506)
11 Ttest_indResult(statistic=-0.514082103706411, pvalue=0.609553346681668)
12 Ttest_indResult(statistic=-0.792642396133211, pvalue=0.4307429990021252)
13 Ttest_indResult(statistic=-0.2810237449585898, pvalue=0.7796922872358806)
14 Ttest

In [75]:
print('1', np.mean(pupilTotal_Difficult_Part1), np.std(pupilTotal_Difficult_Part1))
print('2', np.mean(pupilTotal_Difficult_Part2), np.std(pupilTotal_Difficult_Part2))
print('3', np.mean(pupilTotal_Difficult_Part3), np.std(pupilTotal_Difficult_Part3))
print('4', np.mean(pupilTotal_Difficult_Part4),np.std(pupilTotal_Difficult_Part4))

print('1', np.mean(pupilTotal_Medium_Part1), np.std(pupilTotal_Medium_Part1))
print('2', np.mean(pupilTotal_Medium_Part2), np.std(pupilTotal_Medium_Part2))
print('3', np.mean(pupilTotal_Medium_Part3), np.std(pupilTotal_Medium_Part3))
print('4', np.mean(pupilTotal_Medium_Part4),np.std(pupilTotal_Medium_Part4))


print('1', np.mean(pupilTotal_Easy_Part1), np.std(pupilTotal_Easy_Part1))
print('2', np.mean(pupilTotal_Easy_Part2), np.std(pupilTotal_Easy_Part2))
print('3', np.mean(pupilTotal_Easy_Part3), np.std(pupilTotal_Easy_Part3))
print('4', np.mean(pupilTotal_Easy_Part4),np.std(pupilTotal_Easy_Part4))


1 0.2083060431552143 0.22390721948270248
2 0.16124979254545543 0.22228326594993914
3 0.14810910290462365 0.22910282270814505
4 0.05275096774599053 0.24230720797797184
1 0.29355221149267413 0.312614547340028
2 0.2360379060074025 0.3613892679834723
3 0.17407881649725532 0.3648729587345165
4 0.15426326263088508 0.30160590284738154
1 0.20503701312088835 0.26537754788858653
2 0.22746305733573255 0.3380143060914249
3 0.15521826799415114 0.3277270761050492
4 0.09539992614635055 0.3049653570680041


In [74]:
print(np.mean(pupilTotal_Difficult_Part1), np.std(pupilTotal_Difficult_Part1))

0.2083060431552143 0.22390721948270248


In [103]:
pupilMean = [ np.mean(pupilTotal_Easy), np.mean(pupilTotal_Medium) , np.mean(pupilTotal_Difficult)]
pupilStd = [ np.std(pupilTotal_Easy), np.std(pupilTotal_Medium) , np.std(pupilTotal_Difficult)]

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.errorbar([1, 2, 3], pupilMean, pupilStd, marker = 'o', capsize=4)
plt.xticks([1,2,3], ['Easy', 'Medium', 'Difficult'])

([<matplotlib.axis.XTick at 0x1aca3ec0208>,
 <a list of 3 Text xticklabel objects>)

In [41]:
pupilTotal_Difficult_Part1 = list()
pupilTotal_Medium_Part1 = list()
pupilTotal_Easy_Part1 = list()

pupilTotal_Difficult_Part2 = list()
pupilTotal_Medium_Part2 = list()
pupilTotal_Easy_Part2 = list()

pupilTotal_Difficult_Part3 = list()
pupilTotal_Medium_Part3 = list()
pupilTotal_Easy_Part3 = list()

pupilTotal_Difficult_Part4 = list()
pupilTotal_Medium_Part4 = list()
pupilTotal_Easy_Part4 = list()

In [106]:
np.corrcoef(pupilAvg, scoresDifficulty)

array([[1.        , 0.22915668],
       [0.22915668, 1.        ]])

In [ ]:
scoresDifficultyPlot = [i/100 for i in scoresDifficulty]
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

ax.plot(range(1,len(pupilAvg)+1), pupilR_Avg, 'b')

ax.plot(range(1, len(scoresDifficulty)+1), scoresDifficultyPlot, 'r')

In [ ]:
ipaMean = [ np.mean(ipaEasy), np.mean(ipaMedium) , np.mean(ipaDifficult)]
ipaStd = [ np.std(ipaEasy), np.std(ipaMedium) , np.std(ipaDifficult)]

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.errorbar([1, 2, 3], ipaMean, ipaStd, marker = 'o', capsize=4)
ax.set_title(subjName)
plt.xticks([1,2,3], ['Easy', 'Medium', 'Difficult'])

In [ ]:
print(len(ipaEasy), len(ipaMedium), len(ipaDifficult))

In [ ]:
print('Difficult', np.mean(ipaDifficult), np.std(ipaDifficult))
print('Medium', np.mean(ipaMedium), np.std(ipaMedium))
print('Easy', np.mean(ipaEasy), np.std(ipaEasy))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.errorbar([1, 2, 3], ipaMean, ipaStd, marker = 'o', capsize=4)
plt.xticks([1,2,3], ['Easy', 'Medium', 'Difficult'])

In [40]:
pupil_Difficult_Part1

[0.5367295633713159, 0.17908971232473003]